<a href="https://colab.research.google.com/github/freida20git/SubjectiveQA-Rater/blob/main/notebooks/BERT_Multi_Head.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No hidden layer Model:

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#file_path = '/content/drive/MyDrive/colab/filtered_data - filtered_data.csv'


# Load your dataset
import pandas as pd

#for local run:
file_path = '/workspace/Levona/Noa/NLP_course/filtered_data.csv'
df = pd.read_csv(file_path)

print("Shape:", df.shape)
df.head()


#Multi-head definition
import torch
import torch.nn as nn
from transformers import AutoModel

class MultiHeadBertRegressor(nn.Module):
    def __init__(self, model_name="bert-base-uncased", num_metrics=30, hidden_dim=256):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_metrics)
        ])

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(outputs.pooler_output)  # (batch_size, hidden_size)

        preds = [head(pooled_output) for head in self.heads]  # list of (batch_size, 1)
        preds = torch.cat(preds, dim=1)  # (batch_size, 30)

        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(preds, labels)
            return {"loss": loss, "logits": preds}
        return {"logits": preds}


#Dataset, train and eval:
from torch.utils.data import Dataset, DataLoader

class TextRegressionDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_length=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item

def train_model(model,dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs["loss"]


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].cpu().numpy()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = outputs["logits"].cpu().numpy()

            all_preds.append(preds)
            all_labels.append(labels)

    return torch.vstack([torch.tensor(x) for x in all_labels]).numpy(), torch.vstack([torch.tensor(x) for x in all_preds]).numpy()

#metrics and graphs:
import numpy as np
from scipy.stats import pearsonr, spearmanr
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_predictions(y_true, y_pred):
    num_targets = y_true.shape[1]
    results = {'MSE': [], 'Pearson': [], 'Spearman': []}

    for i in range(num_targets):
        results['MSE'].append(np.mean((y_true[:, i] - y_pred[:, i]) ** 2))
        results['Pearson'].append(pearsonr(y_true[:, i], y_pred[:, i])[0])
        results['Spearman'].append(spearmanr(y_true[:, i], y_pred[:, i])[0])

    return results

def results_to_dataframe(results):
    df = pd.DataFrame(results)
    df['Target'] = [f'Target_{i+1}' for i in range(len(df))]
    return df

def plot_metrics(df_results):
    df_melt = df_results.melt(id_vars='Target', var_name='Metric', value_name='Value')
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_melt, x='Target', y='Value', hue='Metric')
    plt.title("ביצועי מודל לפי מדדים")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

#data loading and run:
import pandas as pd
from transformers import AutoTokenizer
import torch

MODEL_NAME = 'bert-base-uncased'
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 128


file_path = '/workspace/Levona/Noa/NLP_course/filtered_data.csv'
df = pd.read_csv(file_path)

df['full_text'] = df['question_title'].astype(str) + ' ' + df['question_body'].astype(str) + ' ' + df['answer'].astype(str)
text_col = 'full_text'

target_cols = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
    'answer_helpful',
    'answer_level_of_information',
    'answer_plausible',
    'answer_relevance',
    'answer_satisfaction',
    'answer_type_instructions',
    'answer_type_procedure',
    'answer_type_reason_explanation',
    'answer_well_written'
]



# Tokenizer ו-Dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
df[target_cols] = df[target_cols].apply(pd.to_numeric, errors='coerce')

from sklearn.model_selection import train_test_split

texts = df[text_col].tolist()
targets = df[target_cols].values.astype(np.float32)
train_texts, test_texts, train_targets, test_targets = train_test_split(
    texts, targets, test_size=0.2, random_state=42
)

train_dataset = TextRegressionDataset(train_texts, train_targets, tokenizer, max_length=MAX_LEN)
test_dataset = TextRegressionDataset(test_texts, test_targets, tokenizer, max_length=MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


# training:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiHeadBertRegressor(model_name=MODEL_NAME, num_metrics=len(target_cols)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

print("🚀 Starting Training...")
for epoch in range(NUM_EPOCHS):
    loss = train_model(model, train_loader, optimizer, device)
    print(f"Epoch {epoch+1} - Loss: {loss:.4f}")

print("📊 Evaluating...")
y_true, y_pred = evaluate_model(model,  test_loader, device)
results = evaluate_predictions(y_true, y_pred)
df_results = results_to_dataframe(results)
print(df_results)
plot_metrics(df_results)


Including Hidden layer:

In [ ]:
import pandas as pd


# Load your dataset
import pandas as pd

file_path = '/workspace/Levona/Noa/NLP_course/filtered_data.csv'
df = pd.read_csv(file_path)

print("Shape:", df.shape)
df.head()


# Multi head deffinition:
import torch
import torch.nn as nn
from transformers import AutoModel

class MultiHeadBertRegressor(nn.Module):
    def __init__(self, model_name="bert-base-uncased", num_metrics=30, hidden_dim=256):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_metrics)
        ])

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(outputs.pooler_output)  # (batch_size, hidden_size)

        preds = [head(pooled_output) for head in self.heads]  # list of (batch_size, 1)
        preds = torch.cat(preds, dim=1)  # (batch_size, 30)

        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(preds, labels)
            return {"loss": loss, "logits": preds}
        return {"logits": preds}


#Dataset, training and eval
from torch.utils.data import Dataset, DataLoader

class TextRegressionDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_length=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item

def train_model(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs["loss"]


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].cpu().numpy()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = outputs["logits"].cpu().numpy()

            all_preds.append(preds)
            all_labels.append(labels)

    return torch.vstack([torch.tensor(x) for x in all_labels]).numpy(), torch.vstack([torch.tensor(x) for x in all_preds]).numpy()

import numpy as np
from scipy.stats import pearsonr, spearmanr
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_predictions(y_true, y_pred):
    num_targets = y_true.shape[1]
    results = {'MSE': [], 'Pearson': [], 'Spearman': []}

    for i in range(num_targets):
        results['MSE'].append(np.mean((y_true[:, i] - y_pred[:, i]) ** 2))
        results['Pearson'].append(pearsonr(y_true[:, i], y_pred[:, i])[0])
        results['Spearman'].append(spearmanr(y_true[:, i], y_pred[:, i])[0])
    return results

def results_to_dataframe(results):
    df = pd.DataFrame(results)
    df['Target'] = [f'Target_{i+1}' for i in range(len(df))]
    return df

def plot_metrics(df_results):
    df_melt = df_results.melt(id_vars='Target', var_name='Metric', value_name='Value')
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_melt, x='Target', y='Value', hue='Metric')
    plt.title("per metric evaluation:")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

# load and run training:
import pandas as pd
from transformers import AutoTokenizer
import torch

MODEL_NAME = 'bert-base-uncased'
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 128


file_path = '/workspace/Levona/Noa/NLP_course/filtered_data.csv'
df = pd.read_csv(file_path)

df['full_text'] = df['question_title'].astype(str) + ' ' + df['question_body'].astype(str) + ' ' + df['answer'].astype(str)
text_col = 'full_text'


target_cols = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
    'answer_helpful',
    'answer_level_of_information',
    'answer_plausible',
    'answer_relevance',
    'answer_satisfaction',
    'answer_type_instructions',
    'answer_type_procedure',
    'answer_type_reason_explanation',
    'answer_well_written'
]



# Tokenizer ו-Dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
df[target_cols] = df[target_cols].apply(pd.to_numeric, errors='coerce')

df[target_cols] = df[target_cols].apply(pd.to_numeric, errors='coerce')
df = df[df[target_cols].notna().sum(axis=1) >= 25]


from sklearn.model_selection import train_test_split

texts = df[text_col].tolist()
targets = df[target_cols].values.astype(np.float32)

train_texts, test_texts, train_targets, test_targets = train_test_split(
    texts, targets, test_size=0.2, random_state=42
)

train_dataset = TextRegressionDataset(train_texts, train_targets, tokenizer, max_length=MAX_LEN)
test_dataset = TextRegressionDataset(test_texts, test_targets, tokenizer, max_length=MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiHeadBertRegressor(model_name=MODEL_NAME, num_metrics=len(target_cols)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

print("🚀 Starting Training...")
for epoch in range(NUM_EPOCHS):
    loss = train_model(model, train_loader, optimizer, device)
    print(f"Epoch {epoch+1} - Loss: {loss:.4f}")

print("📊 Evaluating...")
y_true, y_pred = evaluate_model(model, test_loader, device)
results = evaluate_predictions(y_true, y_pred)
df_results = results_to_dataframe(results)
print(df_results)
plot_metrics(df_results)
